# Importing the libraries

In [17]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Importing the dataset

In [5]:
movies_df = pd.read_csv('../bm/ml-1m/movies.dat', sep="::", header=None, engine='python', encoding='latin-1')
users_df = pd.read_csv('../bm/ml-1m/users.dat', sep="::", header=None, engine='python', encoding='latin-1')
ratings_df = pd.read_csv('../bm/ml-1m/ratings.dat', sep="::", header=None, engine='python', encoding='latin-1')

# Preparing the training and test set

In [9]:
training_set = pd.read_csv('../bm/ml-100k/u1.base', delimiter="\t", header=None, engine='python', encoding='latin-1')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('../bm/ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

## Get max number of movies and users

In [10]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

## Convert training/test set into array with users as rows and movies as columns

In [11]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

## Convert into torch tensors

In [12]:
training_set = torch.Tensor(training_set)
test_set = torch.Tensor(test_set)

# Creating the neural network architecture

- We are going to use Stacked AutoEncoders.
- Create another class using inheritance to use the variables and the functions from the parent class module

In [15]:
class SAE(nn.Module):
    """
    Contains SAE architecture and helper functions
    """
    def __init__(self):
        super(SAE, self).__init__() # Trigger the base class init function
        # Creating the full connections between input and hidden layer
        self.fc1 = nn.Linear(nb_movies, 20)
        # Creating the full connection between first and second hidden layer
        self.fc2 = nn.Linear(20, 10)
        # Creating the full connection for the decoding phase
        self.fc3 = nn.Linear(10, 20)
        # Final full connection
        self.fc4 = nn.Linear(20, nb_movies)
        # Assign the activation function
        self.activation = nn.Sigmoid()
        
    def forward(self, x):
        """
        Function to perform forward propagation(encoding and decoding)
        """
        output_fc1 = self.activation(self.fc1(x))
        output_fc2 = self.activation(self.fc2(output_fc1))
        output_fc3 = self.activation(self.fc3(output_fc2))
        final_output = self.fc4(output_fc3)
        return final_output
        

# Training the SAE

In [16]:
model = SAE()

In [21]:
loss_function = nn.MSELoss()
optimizer = optim.RMSprop(params=model.parameters(), lr=0.01, weight_decay=0.5)

In [22]:
nb_epoch = 200

In [37]:
for i in range(1, nb_epoch+1):
    train_loss = 0
    s = 0.0
    for id_user in range(nb_users):
        input_vec = Variable(training_set[id_user: id_user + 1])
        target = input_vec.clone()
        if torch.sum(target.data > 0) > 0: # Checking for users who have rated atleast one movie
            output = model(input_vec)
            target.require_grad = False
            # Implement backprop
            output[target == 0] = 0 # Set the values for the prediction to 0 for those 
                                    # movies that have not been rated. No need to updated these 
                                    # weights. Helps optimize code
            loss = loss_function(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10) # consider all 
                                            # movies that have non zero rating. 
                                            # 1e-10 added to make sure denom is never zero
                                            # Mean_corrector represents avg of error that have
                                            # non-zero rating
            loss.backward() # BACKPROP!!!!!!!!. Decides the director to which the weights will be upadted
            train_loss += loss.data[0]*mean_corrector
            s += 1.0
            optimizer.step() # Decides the amount by which the weights will be updated
    print('epoch: '+str(i)+' loss: '+str(train_loss/s))

epoch: 1 loss: 1.108305105189679
epoch: 2 loss: 1.0985055378219062
epoch: 3 loss: 1.0965005213843748
epoch: 4 loss: 1.094128705922906
epoch: 5 loss: 1.0929039675519292
epoch: 6 loss: 1.0906153169759198
epoch: 7 loss: 1.0911172654434056
epoch: 8 loss: 1.089521743202587
epoch: 9 loss: 1.0897679931752646
epoch: 10 loss: 1.088017195797968
epoch: 11 loss: 1.0888474913036794
epoch: 12 loss: 1.087759428104769
epoch: 13 loss: 1.0875038561579053
epoch: 14 loss: 1.0863453686743603
epoch: 15 loss: 1.0875913779049342
epoch: 16 loss: 1.0856944507807134
epoch: 17 loss: 1.0862358167912962
epoch: 18 loss: 1.086338665124423
epoch: 19 loss: 1.0863118356508659
epoch: 20 loss: 1.0848327463962848
epoch: 21 loss: 1.0855504342310258
epoch: 22 loss: 1.0840997557157437
epoch: 23 loss: 1.0831156936514739
epoch: 24 loss: 1.0789145517147634
epoch: 25 loss: 1.07658559680136
epoch: 26 loss: 1.0701003722942122
epoch: 27 loss: 1.070201369274582
epoch: 28 loss: 1.0597532588274357
epoch: 29 loss: 1.0596189773388967
epo

## Testing the model on the test set

In [45]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user])
    if torch.sum(target.data > 0) > 0:
        output = model(input)
        if id_user == 0:
            print(output)
        target.require_grad = False
        output[target == 0] = 0
        loss = loss_function(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data[0]*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

Variable containing:
 4.1623  3.7004  3.5885  ...   2.2359  3.7317  3.2989
[torch.FloatTensor of size 1x1682]

test loss: 0.953415874701
